In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
from shapely import wkt
import datetime
import math

In [39]:
year = '2021'

### Load 50 cities boundary information

In [40]:
city = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/climate_heatwave/city_heatwave/50_city_boundaries.csv', dtype = {'fips': str})
city = city.loc[:, ~city.columns.str.contains('^Unnamed')]
city.head()

,NAME,Number,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry
0,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...
1,Portland,2,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...
2,Portland,2,city,ME,23,60545,66194,2360545,MULTIPOLYGON (((-7812852.7479 5408986.73269999...
3,San Francisco,3,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...
4,Fresno,4,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -..."


In [41]:
def findCentroid(g):
    p1 = wkt.loads(g)
    return p1.centroid.wkt

In [42]:
city['Centroid'] = city['geometry'].apply(findCentroid)
city.head()

,NAME,Number,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,Centroid
0,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515)
1,Portland,2,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...,POINT (-13653343.564582802 5706500.925646672)
2,Portland,2,city,ME,23,60545,66194,2360545,MULTIPOLYGON (((-7812852.7479 5408986.73269999...,POINT (-7823005.794372279 5415746.936720426)
3,San Francisco,3,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...,POINT (-13630069.83625374 4545004.793672214)
4,Fresno,4,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -...",POINT (-13335422.814275231 4408866.950234318)


In [43]:
def findLon(point):
    strList = point.split(' ')
    lon = strList[1][1:]
    lon = float(lon)
    return lon

def findLat(point):
    strList = point.split(' ')
    lat = strList[2][:-1]
    lat = float(lat)
    return lat

In [44]:
city['X'] = city['Centroid'].apply(findLon)
city['Y'] = city['Centroid'].apply(findLat)
city.head()

,NAME,Number,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,Centroid,X,Y
0,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),-1.361700e+07,6.044244e+06
1,Portland,2,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...,POINT (-13653343.564582802 5706500.925646672),-1.365334e+07,5.706501e+06
2,Portland,2,city,ME,23,60545,66194,2360545,MULTIPOLYGON (((-7812852.7479 5408986.73269999...,POINT (-7823005.794372279 5415746.936720426),-7.823006e+06,5.415747e+06
3,San Francisco,3,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...,POINT (-13630069.83625374 4545004.793672214),-1.363007e+07,4.545005e+06
4,Fresno,4,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -...",POINT (-13335422.814275231 4408866.950234318),-1.333542e+07,4.408867e+06


In [45]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
from pyproj import Proj, transform

# Define the projection for Web Mercator (source projection)
proj_web_mercator = Proj(init='epsg:3857')  # Web Mercator projection

# Define the projection for WGS 84 (destination projection)
proj_wgs84 = Proj(init='epsg:4326')  # WGS 84 projection

# Coordinates in Web Mercator (these are the X and Y values)
x_web_mercator = -13616995.217047472
y_web_mercator = 6044244.378846515

# Convert from Web Mercator to WGS 84
longitude, latitude = transform(proj_web_mercator, proj_wgs84, x_web_mercator, y_web_mercator)

print(f"Longitude: {longitude}, Latitude: {latitude}")


Longitude: -122.32354927256164, Latitude: 47.62228131627873


In [47]:
def findLongitude(x, y):
    proj_web_mercator = Proj(init='epsg:3857')
    proj_wgs84 = Proj(init='epsg:4326')
    longitude, latitude = transform(proj_web_mercator, proj_wgs84, x, y)
    return longitude

In [48]:
def findLatitude(x, y):
    proj_web_mercator = Proj(init='epsg:3857')
    proj_wgs84 = Proj(init='epsg:4326')
    longitude, latitude = transform(proj_web_mercator, proj_wgs84, x, y)
    return latitude

In [49]:
city['longitude'] = city.apply(lambda row: findLongitude(row.X, row.Y), axis = 1)
city.head()

,NAME,Number,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,Centroid,X,Y,longitude
0,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),-1.361700e+07,6.044244e+06,-122.323549
1,Portland,2,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...,POINT (-13653343.564582802 5706500.925646672),-1.365334e+07,5.706501e+06,-122.650072
2,Portland,2,city,ME,23,60545,66194,2360545,MULTIPOLYGON (((-7812852.7479 5408986.73269999...,POINT (-7823005.794372279 5415746.936720426),-7.823006e+06,5.415747e+06,-70.275257
3,San Francisco,3,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...,POINT (-13630069.83625374 4545004.793672214),-1.363007e+07,4.545005e+06,-122.441001
4,Fresno,4,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -...",POINT (-13335422.814275231 4408866.950234318),-1.333542e+07,4.408867e+06,-119.794141


In [50]:
city['latitude'] = city.apply(lambda row: findLatitude(row.X, row.Y), axis = 1)
city.head()

,NAME,Number,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,Centroid,X,Y,longitude,latitude
0,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),-1.361700e+07,6.044244e+06,-122.323549,47.622281
1,Portland,2,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...,POINT (-13653343.564582802 5706500.925646672),-1.365334e+07,5.706501e+06,-122.650072,45.537251
2,Portland,2,city,ME,23,60545,66194,2360545,MULTIPOLYGON (((-7812852.7479 5408986.73269999...,POINT (-7823005.794372279 5415746.936720426),-7.823006e+06,5.415747e+06,-70.275257,43.678003
3,San Francisco,3,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...,POINT (-13630069.83625374 4545004.793672214),-1.363007e+07,4.545005e+06,-122.441001,37.755935
4,Fresno,4,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -...",POINT (-13335422.814275231 4408866.950234318),-1.333542e+07,4.408867e+06,-119.794141,36.782742


### Load temperature data

In [51]:
def convertString(f):
    return str(f).rjust(6, '0')

def getMonth(date):
    return(date.strftime('%b'))

def findFog(F):
    return int(F[0])

def findRain(F):
    return int(F[1])

def findSnow(F):
    return int(F[2])

def findHail(F):
    return int(F[3])

def findThunder(F):
    return int(F[4])

def findTornado(F):
    return int(F[5])

def renameColumns(df):
    df = df.rename(columns = {'STATION':'station_id','NAME':'station_name',
                            'LATITUDE':'station_lat', 'LONGITUDE': 'station_lon',
                            'ELEVATION': 'elevation','DATE': 'date', 'TEMP':
                            'mean_temp', 'MIN': 'min_temp', 'MAX': 'max_temp',
                            'DEWP':'dewpoint', 'SLP': 'sea_level_pressure', 'STP':
                            'station_pressure', 'VISIB': 'visibility', 'WDSP':
                            'wind_speed', 'MXSPD': 'max_wind_speed', 'GUST':
                            'wind_gust', 'PRCP': 'precipitation','PRCP_ATTRIBUTES':
                            'precip_flag', 'YEAR':'year', 'Month': 'month'})
    return df

def seperateFRSHTT(df):
    df['fog'] = df['FRSHTT'].apply(findFog)
    df['rain'] = df['FRSHTT'].apply(findRain)
    df['snow'] = df['FRSHTT'].apply(findSnow)
    df['hail'] = df['FRSHTT'].apply(findHail)
    df['thunder'] = df['FRSHTT'].apply(findThunder)
    df['tornado'] = df['FRSHTT'].apply(findTornado)
    return df



In [52]:
ur_files = ddf.read_csv(r'/global/cfs/cdirs/m1532/Projects_MVP/'+
                            'geospatial/enviorment/HeatwaveData/DailyTemp_full/DailyTemp_full_'
                            + year + '.csv', dtype={'STATION': 'object','FRSHTT': 'object'})
temp = ur_files.compute()
temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
temp['FRSHTT'] = temp['FRSHTT'].apply(convertString)
temp['DATE'] = pd.to_datetime(temp['DATE'])
temp.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,YEAR
0,1001099999,2021-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",23.4,24,15.7,24,...,39.4,26.2,,20.1,,0.04,G,NaN,000000,2021
1,1001099999,2021-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,23,30.1,23,...,25.8,33.8,,20.1,,0.06,G,NaN,011000,2021
2,1001099999,2021-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.0,24,34.3,24,...,24.9,39.9,,32.7,,0.11,G,NaN,010000,2021
3,1001099999,2021-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.4,22,29.8,22,...,56.7,37.9,,33.6,,0.01,G,NaN,010000,2021
4,1001099999,2021-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",27.5,24,19.4,24,...,37.9,35.4,,21.7,,0.00,G,NaN,001000,2021


In [16]:
def convertF(f):
    f = f[:-2]
    return f.rjust(6, '0')

In [53]:
temp['FRSHTT'] = temp['FRSHTT'].apply(convertString)
temp.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,YEAR
0,1001099999,2021-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",23.4,24,15.7,24,...,39.4,26.2,,20.1,,0.04,G,NaN,000000,2021
1,1001099999,2021-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,23,30.1,23,...,25.8,33.8,,20.1,,0.06,G,NaN,011000,2021
2,1001099999,2021-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.0,24,34.3,24,...,24.9,39.9,,32.7,,0.11,G,NaN,010000,2021
3,1001099999,2021-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.4,22,29.8,22,...,56.7,37.9,,33.6,,0.01,G,NaN,010000,2021
4,1001099999,2021-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",27.5,24,19.4,24,...,37.9,35.4,,21.7,,0.00,G,NaN,001000,2021


In [54]:
temp = temp[~temp['DATE'].isna()]

In [55]:
temp['month'] = temp['DATE'].apply(getMonth)
temp = renameColumns(temp)
temp = seperateFRSHTT(temp)
temp.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,TEMP_ATTRIBUTES,dewpoint,DEWP_ATTRIBUTES,...,SNDP,FRSHTT,year,month,fog,rain,snow,hail,thunder,tornado
0,1001099999,2021-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",23.4,24,15.7,24,...,NaN,000000,2021,Jan,0,0,0,0,0,0
1,1001099999,2021-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,23,30.1,23,...,NaN,011000,2021,Jan,0,1,1,0,0,0
2,1001099999,2021-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.0,24,34.3,24,...,NaN,010000,2021,Jan,0,1,0,0,0,0
3,1001099999,2021-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.4,22,29.8,22,...,NaN,010000,2021,Jan,0,1,0,0,0,0
4,1001099999,2021-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",27.5,24,19.4,24,...,NaN,001000,2021,Jan,0,0,1,0,0,0


In [56]:
temp.columns

Index(['station_id', 'date', 'station_lat', 'station_lon', 'elevation',
       'station_name', 'mean_temp', 'TEMP_ATTRIBUTES', 'dewpoint',
       'DEWP_ATTRIBUTES', 'sea_level_pressure', 'SLP_ATTRIBUTES',
       'station_pressure', 'STP_ATTRIBUTES', 'visibility', 'VISIB_ATTRIBUTES',
       'wind_speed', 'WDSP_ATTRIBUTES', 'max_wind_speed', 'wind_gust',
       'max_temp', 'MAX_ATTRIBUTES', 'min_temp', 'MIN_ATTRIBUTES',
       'precipitation', 'precip_flag', 'SNDP', 'FRSHTT', 'year', 'month',
       'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado'],
      dtype='object')

In [57]:
def selectColumns(df):
    df = df[['station_id', 'date', 'station_lat', 'station_lon', 'elevation',
             'station_name', 'mean_temp', 'min_temp', 'max_temp', 'dewpoint',
             'sea_level_pressure', 'station_pressure', 'visibility', 'wind_speed',
             'precipitation', 'fog', 'rain', 'snow','hail', 'thunder', 'tornado',
             'year', 'month']]
    return df

In [58]:
temp = selectColumns(temp)
temp.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,min_temp,max_temp,dewpoint,...,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado,year,month
0,1001099999,2021-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",23.4,20.1,26.2,15.7,...,10.9,0.04,0,0,0,0,0,0,2021,Jan
1,1001099999,2021-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,20.1,33.8,30.1,...,12.5,0.06,0,1,1,0,0,0,2021,Jan
2,1001099999,2021-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.0,32.7,39.9,34.3,...,12.2,0.11,0,1,0,0,0,0,2021,Jan
3,1001099999,2021-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.4,33.6,37.9,29.8,...,23.1,0.01,0,1,0,0,0,0,2021,Jan
4,1001099999,2021-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",27.5,21.7,35.4,19.4,...,12.8,0.00,0,0,1,0,0,0,2021,Jan


In [59]:
temp['station_id'].nunique()

12275

In [60]:
def ID_Coordinate(ID,lat, lon):
    setList = []
    setList.append(ID)
    setList.append(lat)
    setList.append(lon)
    return setList

In [61]:
temp['ID_Coordinate'] = temp.apply(lambda x:ID_Coordinate(x['station_id'], x['station_lat'], x['station_lon']), axis=1)
temp.head()

,station_id,date,station_lat,station_lon,elevation,station_name,mean_temp,min_temp,max_temp,dewpoint,...,precipitation,fog,rain,snow,hail,thunder,tornado,year,month,ID_Coordinate
0,1001099999,2021-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",23.4,20.1,26.2,15.7,...,0.04,0,0,0,0,0,0,2021,Jan,"[1001099999, 70.9333333, -8.6666667]"
1,1001099999,2021-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",31.5,20.1,33.8,30.1,...,0.06,0,1,1,0,0,0,2021,Jan,"[1001099999, 70.9333333, -8.6666667]"
2,1001099999,2021-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.0,32.7,39.9,34.3,...,0.11,0,1,0,0,0,0,2021,Jan,"[1001099999, 70.9333333, -8.6666667]"
3,1001099999,2021-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.4,33.6,37.9,29.8,...,0.01,0,1,0,0,0,0,2021,Jan,"[1001099999, 70.9333333, -8.6666667]"
4,1001099999,2021-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",27.5,21.7,35.4,19.4,...,0.00,0,0,1,0,0,0,2021,Jan,"[1001099999, 70.9333333, -8.6666667]"


In [62]:
def dist(p0, p1):
    return (((p0[0] - p1[0])**2) + ((p0[1] - p1[1])**2))**.5

def findSquare(pointList, lat, lon):
    square = []
    for point in pointList:
        point_lat = point[1]
        point_lon = point[2]
        if ((point_lat < lat + 1)
            and (point_lat > lat - 1)
            and (point_lon < lon + 1)
            and (point_lon > lon - 1)):
            square.append(point)
    return square

In [63]:
def shortestDistance(lat, lon):
    p1 = []
    p1.append(lat)
    p1.append(lon)
    distants = []
    square = findSquare(pointList, lat, lon)
    for point in square:
        p_id = point[0]
        p_lat = point[1]
        p_lon = point[2]
        p2 = []
        p2.append(p_lat)
        p2.append(p_lon)
        distant = dist(p1, p2)
        dist_id = []
        dist_id.append(p_id)
        dist_id.append(distant)
        distants.append(dist_id)
    distantList = []    
    for ele in distants:
        distantList.append(ele[1])
    for item in distants:
        if item[1] == min(distantList):
            return item[0]

In [64]:
pointList = temp['ID_Coordinate'].tolist()

In [65]:
city['NearestStation'] = city.apply(lambda x: shortestDistance(x['latitude'], x['longitude']), axis=1)
city.head()

,NAME,Number,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,Centroid,X,Y,longitude,latitude,NearestStation
0,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),-1.361700e+07,6.044244e+06,-122.323549,47.622281,72793524234
1,Portland,2,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...,POINT (-13653343.564582802 5706500.925646672),-1.365334e+07,5.706501e+06,-122.650072,45.537251,72698024229
2,Portland,2,city,ME,23,60545,66194,2360545,MULTIPOLYGON (((-7812852.7479 5408986.73269999...,POINT (-7823005.794372279 5415746.936720426),-7.823006e+06,5.415747e+06,-70.275257,43.678003,99727599999
3,San Francisco,3,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...,POINT (-13630069.83625374 4545004.793672214),-1.363007e+07,4.545005e+06,-122.441001,37.755935,99401699999
4,Fresno,4,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -...",POINT (-13335422.814275231 4408866.950234318),-1.333542e+07,4.408867e+06,-119.794141,36.782742,72389799999


In [66]:
city['NearestStation'].isna().sum()

0

In [67]:
city = city.rename(columns = {'NearestStation': 'station_id'})
city_temp = city.merge(temp, on = ['station_id'], how = 'inner')
city_temp

,NAME,Number,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,Centroid,...,precipitation,fog,rain,snow,hail,thunder,tornado,year,month,ID_Coordinate
0,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),...,0.02,0,1,0,0,0,0,2021,Jan,"[72793524234, 47.54554, -122.31475]"
1,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),...,0.18,0,1,0,0,0,0,2021,Jan,"[72793524234, 47.54554, -122.31475]"
2,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),...,1.46,0,1,0,0,0,0,2021,Jan,"[72793524234, 47.54554, -122.31475]"
3,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),...,0.16,0,1,0,0,0,0,2021,Jan,"[72793524234, 47.54554, -122.31475]"
4,Seattle,1,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,POINT (-13616995.217047472 6044244.378846515),...,0.10,0,1,0,0,0,0,2021,Jan,"[72793524234, 47.54554, -122.31475]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230,Charlotte,50,city,NC,37,12000,731424,3712000,"POLYGON ((-8989636.2259 4217679.263499998, -89...",POINT (-8997961.872585546 4192186.8270590715),...,0.00,0,0,0,0,0,0,2021,Dec,"[72314013881, 35.22254, -80.95433]"
20231,Charlotte,50,city,NC,37,12000,731424,3712000,"POLYGON ((-8989636.2259 4217679.263499998, -89...",POINT (-8997961.872585546 4192186.8270590715),...,0.00,1,0,0,0,0,0,2021,Dec,"[72314013881, 35.22254, -80.95433]"
20232,Charlotte,50,city,NC,37,12000,731424,3712000,"POLYGON ((-8989636.2259 4217679.263499998, -89...",POINT (-8997961.872585546 4192186.8270590715),...,0.00,0,1,0,0,0,0,2021,Dec,"[72314013881, 35.22254, -80.95433]"
20233,Charlotte,50,city,NC,37,12000,731424,3712000,"POLYGON ((-8989636.2259 4217679.263499998, -89...",POINT (-8997961.872585546 4192186.8270590715),...,0.24,0,1,0,0,0,0,2021,Dec,"[72314013881, 35.22254, -80.95433]"


In [68]:
city_temp.columns

Index(['NAME', 'Number', 'CLASS', 'ST', 'STFIPS', 'PLACEFIPS', 'POP2010',
       'STPLFIPS', 'geometry', 'Centroid', 'X', 'Y', 'longitude', 'latitude',
       'station_id', 'date', 'station_lat', 'station_lon', 'elevation',
       'station_name', 'mean_temp', 'min_temp', 'max_temp', 'dewpoint',
       'sea_level_pressure', 'station_pressure', 'visibility', 'wind_speed',
       'precipitation', 'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado',
       'year', 'month', 'ID_Coordinate'],
      dtype='object')

In [69]:
city_temp = city_temp[['NAME', 'longitude', 'latitude', 'ID_Coordinate', 'year', 'month', 'date', 
                       'elevation', 'mean_temp', 'min_temp', 'max_temp', 'dewpoint', 'sea_level_pressure', 
                       'station_pressure', 'visibility', 'wind_speed', 'precipitation', 'fog', 
                       'rain', 'snow', 'hail', 'thunder', 'tornado']]
city_temp = city_temp.rename(columns = {'NAME': 'city_name'})
city_temp

,city_name,longitude,latitude,ID_Coordinate,year,month,date,elevation,mean_temp,min_temp,...,station_pressure,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado
0,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-01,7.6,47.2,44.1,...,13.2,8.1,5.4,0.02,0,1,0,0,0,0
1,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-02,7.6,48.4,44.1,...,8.2,8.0,12.7,0.18,0,1,0,0,0,0
2,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-03,7.6,43.9,37.9,...,8.7,7.8,9.7,1.46,0,1,0,0,0,0
3,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-04,7.6,45.9,37.9,...,4.8,9.0,6.7,0.16,0,1,0,0,0,0
4,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-05,7.6,44.0,39.9,...,15.2,9.0,8.0,0.10,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230,Charlotte,-80.830067,35.208024,"[72314013881, 35.22254, -80.95433]",2021,Dec,2021-12-27,222.6,60.5,50.0,...,987.4,9.8,4.6,0.00,0,0,0,0,0,0
20231,Charlotte,-80.830067,35.208024,"[72314013881, 35.22254, -80.95433]",2021,Dec,2021-12-28,222.6,61.4,50.0,...,984.8,8.6,6.4,0.00,1,0,0,0,0,0
20232,Charlotte,-80.830067,35.208024,"[72314013881, 35.22254, -80.95433]",2021,Dec,2021-12-29,222.6,66.6,52.0,...,984.7,9.4,11.3,0.00,0,1,0,0,0,0
20233,Charlotte,-80.830067,35.208024,"[72314013881, 35.22254, -80.95433]",2021,Dec,2021-12-30,222.6,64.7,62.1,...,984.7,9.3,7.0,0.24,0,1,0,0,0,0


In [70]:
city_temp = city_temp.rename(columns = {'ID_Coordinate': 'station_coordinate'})
city_temp.head()

,city_name,longitude,latitude,station_coordinate,year,month,date,elevation,mean_temp,min_temp,...,station_pressure,visibility,wind_speed,precipitation,fog,rain,snow,hail,thunder,tornado
0,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-01,7.6,47.2,44.1,...,13.2,8.1,5.4,0.02,0,1,0,0,0,0
1,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-02,7.6,48.4,44.1,...,8.2,8.0,12.7,0.18,0,1,0,0,0,0
2,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-03,7.6,43.9,37.9,...,8.7,7.8,9.7,1.46,0,1,0,0,0,0
3,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-04,7.6,45.9,37.9,...,4.8,9.0,6.7,0.16,0,1,0,0,0,0
4,Seattle,-122.323549,47.622281,"[72793524234, 47.54554, -122.31475]",2021,Jan,2021-01-05,7.6,44.0,39.9,...,15.2,9.0,8.0,0.10,0,1,0,0,0,0


In [71]:
city_temp['city_name'].nunique()

50

In [72]:
year

'2021'

In [73]:
city_temp.to_csv('city_climate/city_climate_' + year + '.csv')